In [ ]:
import sys

sys.path.append("../src")

from fabricengineer.import_module.import_module import import_module

VERSION = "0.0.8"

In [ ]:
code = import_module("transform.silver.insertonly", VERSION)

print(code)

import os

from datetime import datetime
from typing import Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
from uuid import uuid4

from pyspark.sql import (
    SparkSession,
    DataFrame,
    functions as F,
    types as T,
    Window
)


@dataclass
class LakehouseTable:
    lakehouse: str
    schema: str
    table: str

    @property
    def table_path(self) -> str:
        return f"{self.lakehouse}.{self.schema}.{self.table}"


@F.udf(returnType=T.StringType())
def generate_uuid():
    """Generiert eine UUID4"""
    return str(uuid4())


@dataclass(frozen=True)
class ConstantColumn:
    """Class for adding a column with constant value to etl"""
    name: str
    value: str
    part_of_nk: bool = False

    def __post_init__(self):
        """
        Nach initialisierung wird der name in UPPERCASE umgewandelt.
        """
        object.__setattr__(self, "name", self.name.upper())


def get_mock_table_path(table: LakehouseTable) -> str:
    if table 

In [3]:
with open("./code.py", "w") as f:
    f.write(code)

In [4]:
exec(code)

In [7]:
etl

In [8]:
table = LakehouseTable(
    lakehouse="test_lakehouse",
    schema="test_schema",
    table="test_table"
)
table

LakehouseTable(lakehouse='test_lakehouse', schema='test_schema', table='test_table')

In [3]:
code = import_module("transform.silver.mlv", VERSION)

print(code)

ValueError: ('Module content is malformed: https://raw.githubusercontent.com/enricogoerlitz/fabricengineer-py/refs/tags/0.0.7/src/fabricengineer/transform/mlv/mlv.py.', 'Content: from typing import Any\nfrom pyspark.sql import DataFrame, SparkSession\n\n\nclass MaterializedLakeView:\n    def __init__(\n        self,\n        lakehouse: str = None,\n        schema: str = None,\n        table: str = None,\n        table_suffix: str = "_mlv",\n        spark_: SparkSession = None,\n        notebookutils_: Any = None,\n        is_testing_mock: bool = False\n    ) -> None:\n        self.init(\n            lakehouse=lakehouse,\n            schema=schema,\n            table=table,\n            table_suffix=table_suffix,\n            spark_=spark_,\n            notebookutils_=notebookutils_,\n            is_testing_mock=is_testing_mock\n        )\n\n    def init(\n        self,\n        lakehouse: str,\n        schema: str,\n        table: str,\n        table_suffix: str = "_mlv",\n        spar